In [1]:
import numpy as np
import pandas as pd
from math import inf

In [2]:
# Separates the sequences line by line
def convert_to_seq(ar):
    dna = []
    number_of_items = 0
    for i in range(ar.shape[0]):
        if ar[i][0] != ">":
            dna.append([ar[i][j] for j in range(len(ar[i]))])
        else:
            number_of_items += 1
    return np.array(dna, dtype=object), number_of_items


# combines all lines of the same sequence
def form_complete_seq(dna, n):
    final = []
    start = 0
    inc = int(len(dna) / n)
    end = inc

    while end <= dna.shape[0]:
        temp = []
        for i in range(start, end):
            for j in range(len(dna[i])):
                temp.append(dna[i][j])
        temp = np.array(temp)
        final.append(temp)
        start = start + inc
        end = end + inc

    return np.array(final)


# calculates score for using BLOSUM62.txt matrix
def find_score(a, b, dmat):
    x = 0
    y = 0
    if a == "-":
        a = "*"
    if b == "-":
        b = "*"
    
    if (a == "-" and b != "-") or (a != "-" and b == "-"):
        return (0, True)
        
    
    for i in range(dmat.shape[0]):
        if a == dmat[0][i]:
            x = i
        if b == dmat[0][i]:
            y = i
    return (dmat[x][y], False)


# finds the similarity between two given sequences
# using BLOSUM62.txt values
def find_dist2(m, n, final, dmat):
    # here m, n = seq1, seq2
    count = 0
    isprev = False

    for i in range(len(final[m])):
        temp, iscurrent = find_score(final[m][i], final[n][i], dmat)
        if isprev == False and iscurrent == True:
            count -= 11
            isprev = True
        elif isprev == True and iscurrent == True:
            count -= 1
        elif iscurrent == False:
            count += int(temp)
            isprev = False
    return count


# finds the similarity between two given sequences 
def find_dist(m, n, final):
    count = 0
    
    for i in range(len(final[0])):
        if final[m][i] != final[n][i]:
            count += 1     
    count /= len(final[0])
    return count


# for task 2
def generate_dist_matrix2(final, k, dmat):
    mat = [[0 for i in range(k)] for j in range(k)]
    
    # here x, y = seq1, seq2
    for x in range(k):
        for y in range(k):
            mat[x][y] = find_dist2(x, y, final, dmat)
            
    return np.array(mat)


# for task 1
def generate_dist_matrix(final, k):
    mat = [[0 for i in range(k)] for j in range(k)]

    # here x, y = seq1, seq2
    for x in range(k):
        for y in range(k):
            mat[x][y] = find_dist(x, y, final)
            
    return np.array(mat)

In [3]:
# copies the content from the dataframe - mat 
# to an array that it creates - new_mat
def copy_to_arr(mat):
    new_mat = np.array([[0.0 for i in range(len(mat))] for j in range(len(mat))])
    for i in range(len(mat)):
        for j in range(len(mat)):
            if i > j:
                new_mat[i][j] = mat.iloc[i, j]
    return (new_mat)


# finds the min value using the matrix it generated above
def find_min(new_mat):
        x = 0
        y = 0
        minVal = inf

        for i in range(new_mat.shape[0]):
            for j in range(i+1):
                if new_mat[i][j] <= minVal and new_mat[i][j] != 0:
                    minVal = new_mat[i][j]
                    x = i
                    y = j
                    
        return (x, y, minVal)
    

# finds the min value using the matrix it generated above
def find_max(new_mat):
        x = 0
        y = 0
        maxVal = -inf

        for i in range(new_mat.shape[0]):
            for j in range(i+1):
                if new_mat[i][j] >= maxVal and new_mat[i][j] != 0:
                    maxVal = new_mat[i][j]
                    x = i
                    y = j
                    
        return (x, y, maxVal)
    
    
# deletes the two rows and columns and generates a new columnn
# in the place of the min(row, col)
# using UPGMA method
def upgma(new_mat, x, y):
    pivot = min(x, y)
    for j in range(0, pivot):
        if new_mat[max(x, y)][j] != 0:
            new_mat[pivot][j] = (new_mat[pivot][j] + new_mat[max(x, y)][j]) / 2
        else:
            new_mat[pivot][j] = (new_mat[pivot][j] + new_mat[j][max(x, y)]) / 2
    
    for i in range(pivot+1, new_mat.shape[0]):
        if new_mat[max(x, y)][i] != 0:
            new_mat[i][pivot] = (new_mat[i][pivot] + new_mat[max(x, y)][i]) / 2
        else:
            new_mat[i][pivot] = (new_mat[i][pivot] + new_mat[i][max(x, y)]) / 2
                                 
    return (new_mat, min(x, y), max(x, y))


# after the above calculations
# it copies the contents of the array - new_mat
# into the datafram - mat
def copy_to_df(new_mat, mat):              
    for i in range(new_mat.shape[0]):
        for j in range(new_mat.shape[0]):
                mat.iloc[i, j] = new_mat[i][j]
    return (mat)


# it drops the higher index row and column
# i.e max(x, y)
def drop_extra(mat, drop_val):
    temp = mat.drop([str(drop_val)], axis = 1)
    temp = temp.drop([str(drop_val)])
    return(temp)


# we now have a matrix of order n-1*n-1
# renames the merged rows, columns
def rename_merged(temp, merge_val, drop_val):
    temp = temp.rename(columns={str(merge_val): str(merge_val)+','+str(drop_val)})
    temp = temp.rename(index={str(merge_val): str(merge_val)+','+str(drop_val)})
    return (temp)

In [4]:
# automates the process sequentially 
def to_do(mat, ans, part):
    
    # copies data from the dataframe
    # into a 2D numpy array
    new_mat = copy_to_arr(mat)

    # finds min value in part-1
    # finds max value in part-2
    # the variable is called min_val in both cases for ease
    if part == 1:
        x, y, min_val = find_min(new_mat)
    elif part == 2:
        x, y, min_val = find_max(new_mat)

    # actual logic of UPGMA runs here
    new_mat, merge_val, drop_val = upgma(new_mat, x, y)

    # after calculations the info is 
    # copied back into the dataframe
    # finds index of row, col to be dropped
    mat = copy_to_df(new_mat, mat)
    merge_val = mat.columns[merge_val]
    drop_val = mat.columns[drop_val]
    
    # ans contains the clubbing at every step
    ans.append([merge_val, drop_val, "{0:.3f}".format(min_val/2)])

    # order of the dataframe matrix
    # is reduced
    mat = drop_extra(mat, drop_val)
    mat = rename_merged(mat, merge_val, drop_val)
    return (mat, ans)

In [5]:
def part_1():
    
    # loads the information from files
    fname = "Nucleotide_alignment.txt"
    ar = np.loadtxt(fname, delimiter="\t", dtype='str')
    
    # generates the final distance output
    dna, n = convert_to_seq(ar)
    final = form_complete_seq(dna, n)
    mat = generate_dist_matrix(final, n)
    
    # converts it to a dictionary 
    # and exports it as a csv file
    res = {idx + 1 : mat[idx] for idx in range(len(mat))} 
    file_name = "Ndistance.txt"
    pd.DataFrame(res).to_csv(file_name)
    
    return (file_name)

In [6]:
def part_2():
    
    # loads the information from files
    fname = "Protein_alignment.txt"
    fname2 = "BLOSUM62.txt"
    ar = np.loadtxt(fname, delimiter="\t", dtype='str')
    ar2 = np.loadtxt(fname2, dtype='str')
    
    # dmat is the input distance matrix
    # from BLOSUM62.txt
    dmat, m = convert_to_seq(ar2)

    # generates the final distance output
    protein_seq, n = convert_to_seq(ar)
    final = form_complete_seq(protein_seq, n)
    dist_matrix = generate_dist_matrix2(final, n, dmat)

    # converts it to a dictionary 
    # and exports it as a csv file
    res = {idx + 1 : dist_matrix[idx] for idx in range(len(dist_matrix))} 
    file_name = "Pdistance.txt"
    pd.DataFrame(res).to_csv(file_name)
    
    return (file_name)

In [7]:
# generates the phylogenetic relationship 
def gen_tree(fname, part):
    
    mat = pd.read_csv(fname)
    
    # manipulates the names of rows and columns
    mat = mat.drop(['Unnamed: 0'], axis=1)
    for col in mat.columns: 
        mat = mat.rename(columns={col : str(int(col)-1)})
    disp_mat = mat
    for idx in mat.index:
        mat = mat.rename(index={idx : str(idx)})
    
    # ans stores step by step clubbing 
    # of two columns
    ans = []
    while len(mat) > 1:
        mat, ans = to_do(mat, ans, part)
    
    return(ans, disp_mat)

In [8]:
# formats it to the Newick form
def convert_to_newick(ans):
    new_dict = {}

    for row in ans:
        temp = row[0] + "," + row[1]
        if row[0] not in new_dict and row[1] not in new_dict:
            new_dict[temp] = "(" + temp + ": " + str(row[2]) + ")"
        if row[0] in new_dict and row[1] not in new_dict:
            new_dict[temp] = "(" + new_dict[row[0]] + ', ' + row[1] + ': ' + str(row[2]) + ')'
        if row[1] in new_dict and row[0] not in new_dict:
            new_dict[temp] = "(" + new_dict[row[1]] + ', ' + row[0] + ': ' + str(row[2]) + ')'
        if row[0] in new_dict and row[1] in new_dict:
            new_dict[temp] = "(" + new_dict[row[0]] + ", " + new_dict[row[1]] + ': ' + str(row[2]) + ")"

    count = 0
    for x in new_dict:
        if count == len(new_dict)-1:
            return (new_dict[x])
        count += 1

    return -1

In [9]:
# saves the final answer in a file
def export_answer(fname, content):
    file = open(fname, "w")
    file.write(content)
    file.close()

# Task - 1

In [10]:
ans1, mat1 = gen_tree(part_1(), 1)


In [11]:
mat1

,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.573975,0.673797,0.645276,0.632799,0.629234,0.650624,0.609626,0.677362,0.673797
1,0.573975,0.000000,0.554367,0.527629,0.515152,0.527629,0.515152,0.502674,0.573975,0.541889
2,0.673797,0.554367,0.000000,0.383244,0.376114,0.333333,0.354724,0.386809,0.363636,0.351159
3,0.645276,0.527629,0.383244,0.000000,0.322638,0.352941,0.222816,0.352941,0.327986,0.253119
4,0.632799,0.515152,0.376114,0.322638,0.000000,0.326203,0.301248,0.360071,0.345811,0.286988
5,0.629234,0.527629,0.333333,0.352941,0.326203,0.000000,0.324421,0.270945,0.281640,0.335116
6,0.650624,0.515152,0.354724,0.222816,0.301248,0.324421,0.000000,0.311943,0.288770,0.229947
7,0.609626,0.502674,0.386809,0.352941,0.360071,0.270945,0.311943,0.000000,0.311943,0.342246
8,0.677362,0.573975,0.363636,0.327986,0.345811,0.281640,0.288770,0.311943,0.000000,0.158645
9,0.673797,0.541889,0.351159,0.253119,0.286988,0.335116,0.229947,0.342246,0.158645,0.000000


In [12]:
c1 = convert_to_newick(ans1)
c1

'(((((((3,6: 0.111), (8,9: 0.079): 0.137), 4: 0.157), (5,7: 0.135): 0.167), 2: 0.182), 1: 0.269), 0: 0.307)'

In [13]:
export_answer("Ndistance_newick_form.txt", c1)

# Task - 2

In [14]:
ans2, mat2 = gen_tree(part_2(), 2)


In [15]:
mat2

,0,1,2,3,4,5,6,7,8,9
0,589,168,255,221,201,200,194,212,210,214
1,168,591,237,332,314,270,332,347,359,356
2,255,237,579,331,346,333,302,331,330,326
3,221,332,331,596,459,425,444,468,485,482
4,201,314,346,459,583,499,423,480,487,484
5,200,270,333,425,499,575,388,451,442,438
6,194,332,302,444,423,388,575,455,481,478
7,212,347,331,468,480,451,455,587,522,518
8,210,359,330,485,487,442,481,522,591,584
9,214,356,326,482,484,438,478,518,584,591


In [16]:
c2 = convert_to_newick(ans2)
c2

'((((((((8,9: 292.000), 7: 260.000), 3: 237.875), 6: 227.812), (4,5: 249.500): 214.641), 2: 163.906), 1: 137.883), 0: 99.234)'

In [17]:
export_answer("Pdistance_newick_form.txt", c2)